# Lab3 Acquisition

本实验用于演示：
1. 建立`CrackerS1`测试设备采集`Nut_stm32f103c8`的功耗信号；
2. 为`CrackerS1`供电；
3. 建立`CrackerS1`同`CrackNuts`控制分析软件的连接。

## 准备工作

硬件：
1. `CrackerS1`侧信道分析设备；
2. `Nut_stm32f103c8`测试对象；
3. CAT6网线2~3条；
4. POE交换机、POE电源适配器或12V直流适配器；
5. 无线路由器（可选）。

软件：
1. 安装最新[`CrackNuts`](https://cracknuts.cn/docs/doc/getting-started/host_sdk_installation)控制分析软件；
2. `Nut_stm32f103c8`Golden固件程序；
3. 安装[STM32 ST-LINK utility](https://www.st.com/en/development-tools/stsw-link004.html#get-software)固件下载程序。

## 实验步骤

1. 在`Nut_stm32f103c8`测试板中下载Golden固件程序。
2. 将`Nut_stm32f103c8`测试板插入`CrackerS1`的Nut Socket中。
3. 电脑连接`CrackerS1`测试设备，参考[Lab1_Setup](Lab1_Setup.ipynb)。
4. 调整采样参数。
5. 开始采集。

### 引入依赖包

In [7]:
import cracknuts as cn  # 导入cracknuts包
from cracknuts.cracker import serial  # 串行协议参数
from cracknuts.cracker.cracker_s1 import CrackerS1  # 导入Cracker-S1

s1 = CrackerS1(address='192.168.0.10')  # 具体地址根据OLED显示IP修改
s1.connect()

### 接口配置

In [8]:
from enum import Enum

class AlgType(Enum):  # Algrithom type
    AES_ENC = 1
    AES_DEC = 2
    DES_ENC = 3
    DES_DEC = 4

class IntfType(Enum):  # interface type
    UART = 1
    SPI = 2 # 完善中
    I2C = 3 # 完善中
    CAN = 4 # 完善中

In [9]:
# Algorithm select
alg = AlgType.AES_ENC # 修改这个值改变算法
# alg = AlgType.AES_DEC # 修改这个值改变算法
# alg = AlgType.DES_ENC # 修改这个值改变算法
# alg = AlgType.DES_DEC # 修改这个值改变算法

# data random
# fix_data = True
fix_data = False

# interface
intf = IntfType.UART
# intf = IntfType.SPI
# intf = IntfType.I2C
# intf = IntfType.CAN

### Nut 命令

In [10]:
# AES Command
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_set_aes_dec_key = "01 01 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"
cmd_aes_dec = "01 03 00 00 00 00 00 10"
aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

# DES Command
cmd_set_des_enc_key = "02 00 00 00 00 00 00 08"
cmd_set_des_dec_key = "02 01 00 00 00 00 00 08"
cmd_des_enc = "02 02 00 00 00 00 00 08"
cmd_des_dec = "02 03 00 00 00 00 00 08"
des_key = "12 34 56 78 90 ab cd ef"
des_data_len = 8

### 核心采集流程

In [11]:
import random
import time

def init(c):        
    if intf is IntfType.UART:
        s1.uart_enable()
    elif intf is IntfType.SPI:
        s1.spi_enable()
        s1.spi_config(speed=1000, cpol=serial.SpiCpol.SPI_CPOL_LOW, cpha=serial.SpiCpha.SPI_CPHA_LOW)
    
    if alg is AlgType.AES_ENC:
        cmd = cmd_set_aes_enc_key + aes_key
    elif alg is AlgType.AES_DEC:
        cmd = cmd_set_aes_dec_key + aes_key
    elif alg is AlgType.DES_ENC:
        cmd = cmd_set_des_enc_key + des_key
    else:
        cmd = cmd_set_des_dec_key + des_key

    if intf is IntfType.UART:
        status, ret = s1.uart_transmit_receive(cmd, timeout=1000, rx_count=6)
    elif intf is IntfType.SPI:
        status, ret = s1.spi_transmit(tx_data=cmd_set_aes_enc_key, is_trigger=False)
        status, ret = s1.spi_transmit(tx_data=aes_key, is_trigger=False)
        # time.sleep(0.5)
        status, ret = s1.spi_receive(rx_count=6)

def do(c):
    
    if alg is AlgType.AES_ENC:
        cmd = cmd_aes_enc
        data_len = aes_data_len
    elif alg is AlgType.AES_DEC:
        cmd = cmd_aes_dec
        data_len = aes_data_len
    elif alg is AlgType.DES_ENC:
        cmd = cmd_des_enc
        data_len = des_data_len
    else:
        cmd = cmd_des_dec
        data_len = des_data_len

    # convert cmd to bytes
    cmd = cmd.replace(' ', '')
    cmd = bytes.fromhex(cmd)

    # concate payload
    if fix_data: ## fix data
        data = bytes(data_len)
    else:  # random data
        if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
            data = random.randbytes(aes_data_len)
        else:
            data = random.randbytes(des_data_len)
    
    d = cmd + data
    # print(d.hex())

    if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
        if intf is IntfType.UART:
            # print(d)
            status, ret = s1.uart_transmit_receive(d, rx_count= 6 + aes_data_len, is_trigger=True)
            # print(ret)
        elif intf is IntfType.SPI:
            status, ret = s1.spi_transmit(tx_data=cmd, is_trigger=False)
            status, ret = s1.spi_transmit(tx_data=data, is_trigger=True)
            time.sleep(0.5)
            status, ret = s1.spi_receive(rx_count=6 + aes_data_len)
        # ret = cracker.spi_receive(12 + aes_data_len)
        return {'plaintext': data, 'ciphertext': ret[-aes_data_len:]}
    else:
        # _, ret = cracker.cracker_serial_data(6 + des_data_len, d)
        if intf is IntfType.UART:
            status, ret = s1.uart_transmit_receive(d, rx_count=6 + des_data_len, is_trigger=True)
            # print(ret)
        elif intf is IntfType.SPI:
            status, ret = s1.spi_transmit(tx_data=cmd, is_trigger=False)
            status, ret = s1.spi_transmit(tx_data=data, is_trigger=True)
            time.sleep(0.1)
            status, ret = s1.spi_receive(rx_count=6 + des_data_len)
        # cracker.spi_transmit(d)
        # ret = cracker.spi_receive(6 + des_data_len)
        # print(len(ret))
        # print(ret.hex())
        # print(ret[-des_data_len:].hex())
        return {'plaintext': data, 'ciphertext': ret[-des_data_len:]}

saveDataLength = 2*aes_data_len if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC else 2*des_data_len
acq = cn.new_acquisition(s1, do=do, init= init)

### 采集界面

In [12]:
cn.panel(acq)

[WARNING] 2025-05-21 14:15:26,325 cracknuts_panel.__init__:62 The configuration item osc_channel_0_gain differs between the configuration file (80) and the cracker (10).


CracknutsPanelWidget(acq_run_progress={'finished': 0, 'total': -1}, connect_status=True, custom_y_range={'0': …

以上展示`Cracker`的面板后，可以使用测试模式调节Nut、SCOPE采样等参数达到合适的效果，然后使用运行开始正式采集。